In [31]:
import os, sys
import re
from dotenv import load_dotenv
import time
import pandas as pd
from openai import AzureOpenAI, AsyncAzureOpenAI
import tiktoken
import asyncio

In [2]:
load_dotenv('.env')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY', None)

In [3]:
token_encoder = tiktoken.get_encoding('cl100k_base')
def count_tokens(txt):
    return len(token_encoder.encode(txt))

In [4]:
dbt_skills_ref = pd.read_csv(os.path.join('..', 'data', 'dbt_skills_ref.csv'))
dbt_skills_ref

,skill_id,skill_name,skill_desc,module_focus,module_name,category_name
0,M1,Wise Mind,"- The ""Wise Mind"" skill in the Mindfulness mod...",Acceptance,Mindfulness,NaN
1,M2,Observing,"- The ""Observing"" skill in the Mindfulness mod...",Acceptance,Mindfulness,"Mindfulness ""What"" Skills"
2,M3,Describing,"- The ""Describing"" skill in the Mindfulness mo...",Acceptance,Mindfulness,"Mindfulness ""What"" Skills"
3,M4,Participating,"- The ""Participating"" skill in the Mindfulness...",Acceptance,Mindfulness,"Mindfulness ""What"" Skills"
4,M5,Nonjudgmentally,"- The ""Nonjudgmentally"" skill in the Mindfulne...",Acceptance,Mindfulness,"Mindfulness ""How"" Skills"
5,M6,One-Mindfully,"- The ""One-Mindfully"" skill in the Mindfulness...",Acceptance,Mindfulness,"Mindfulness ""How"" Skills"
6,M7,Effectively,"- The ""Effectively"" skill in the Mindfulness m...",Acceptance,Mindfulness,"Mindfulness ""How"" Skills"
7,M8,Mindfulness Practice: A Spiritual Perspective,"- The ""Mindfulness Practice: A Spiritual Persp...",Acceptance,Mindfulness,Other Perspectives on Mindfulness
8,M9,Balancing Doing Mind and Being Mind,"- The ""Balancing Doing Mind and Being Mind"" sk...",Acceptance,Mindfulness,Other Perspectives on Mindfulness
9,IE1,Clarifying Priorities,"- The ""Clarifying Priorities"" skill in the Int...",Change,Interpersonal Effectiveness,Obtaining Objectives Skillfully


In [5]:
dbt_skills_ref.skill_desc[0]

'- The "Wise Mind" skill in the Mindfulness module of DBT refers to a state of mind where an individual integrates both rational thought (Reasonable Mind) and emotional experience (Emotion Mind) to make balanced and effective decisions.\n- It is considered a core concept in DBT, representing an inner knowledge and intuition that everyone possesses, which can guide them to a sense of truth and balanced action.\n- In practice, this skill involves recognizing when one is operating from either the Reasonable Mind or the Emotion Mind and then working towards achieving a synthesis of the two, which is the Wise Mind.\n- During a session, it might be identified by the therapist asking the client to reflect on a situation and consider what their emotions, reason, and intuition are telling them, aiming to find a middle path that incorporates all aspects.'

In [6]:
convos = pd.read_pickle(os.path.join('..', 'convo_generation', 'output', '20240306_convos_full.pickle'))

In [7]:
skill_extract_test = convos['bc97e560-2f17-40c6-bbbc-cf1455e5536b']['result']['eval_messages']
print(skill_extract_test)

Therapist: Hello and welcome! I'm glad you're here to explore Dialectical Behavior Therapy (DBT) skills. Is there a particular DBT skill or technique you're interested in practicing today, or would you like me to suggest one based on what you're experiencing currently?

Client: My grandmother passed away two days ago and everyone in my family has been busy grieving so I've been picking up all the preparations for her wake. I haven't had a chance to stop but if I don't take care of everything, I don't think anyone else will.

Therapist: I'm so sorry to hear about your loss. It sounds like you’re carrying a heavy burden during this tough time. Grief can be overwhelming, and it's important to take care of yourself too. How are you currently feeling in this moment as you handle these responsibilities?

Client: Thank you for your kind words. I'm feeling incredibly overwhelmed and exhausted. It's like I'm running on autopilot, trying to keep it all together for my family's sake, but inside I

In [26]:
count_tokens(skill_extract_test)

595

In [9]:
client = AzureOpenAI(
    api_key = OPENAI_API_KEY,
    api_version = "2023-05-15",
    azure_endpoint = "https://dbt-openai-usea2-assistants.openai.azure.com/"
)

In [10]:
client_async = AsyncAzureOpenAI(
    api_key = OPENAI_API_KEY,
    api_version = "2023-05-15",
    azure_endpoint = "https://dbt-openai-usea2-assistants.openai.azure.com/"
)

In [11]:
skill_extract_base_prompt = '''
You are tasked with evaluating whether a therapist discussed the following DBT skill with their client during a session:
<SKILL_DESC>

Carefully analyze the provided conversation log between a therapist and their client, then assign a numerical rating based on the following criteria.
Assign a "1" for "present" if the therapist:
- Explicitly refers to this skill by name.
- Introduces, suggests, defines, or explains this skill to the client.
- Teaches this skill to the client.
- Guides the client in practicing this skill.
- Assists the client in applying this skill to their life.
Assign a "0" for "not present" if the therapist:
- Does not refer to this skill by name.
- Does not introduce, suggest, define, or explain this skill to the client.
- Does not teach this skill to the client.
- Does not guide the client in practicing this skill.
- Does not assist the client in applying this skill to their life.

Instructions:
- Please provide your rating as a standalone numerical value, with no additional comments or justification.
- Valid responses are limited to: [1, 0]
'''.strip()

In [12]:
skill_extract_base_prompt2 = '''
Determine if a therapist discussed the following Dialectical Behavior Therapy (DBT) skill during a session:
<SKILL_DESC>

Carefully review the provided transcript between the therapist and client. Rate whether the therapist discussed the DBT skill using the following criteria:

Rate "1" if the therapist did any of the following:
- Mentioned the skill by name.
- Defined, described, explained, or taught the skill to the client.
- Led the client through guided practice of the skill.
- Helped apply the skill to the client's life.

Rate "0" if none of these actions took place.

Provide your rating as a single number, either "1" or "0", with no further explanation.
'''.strip()

In [19]:
t_start = time.perf_counter()

for row in dbt_skills_ref.itertuples():
    row_sys_prompt = skill_extract_base_prompt2.replace("<SKILL_DESC>", row.skill_desc)
    response = client.chat.completions.create(
        # model="gpt4-1106",
        model="gpt-35-turbo",
        messages=[
            {"role": "system", "content": row_sys_prompt},
            {"role": "user", "content": skill_extract_test}
        ],
        n=5,
        temperature=0.
    )
    ratings = [choice.message.content for choice in response.choices]
    print(f'{row.skill_name}: {ratings}')

t_stop = time.perf_counter()
elapsed_time = t_stop - t_start
print(elapsed_time)

Wise Mind: ['0', '0', '0', '0', '0']
Observing: ['0', '0', '0', '0', '0']
Describing: ['0', '0', '0', '0', '0']
Participating: ['0', '0', '0', '0', '0']
Nonjudgmentally: ['0', '0', '0', '0', '0']
One-Mindfully: ['0', '0', '0', '0', '0']
Effectively: ['0', '0', '0', '0', '0']
Mindfulness Practice: A Spiritual Perspective: ['0', '0', '0', '0', '0']
Balancing Doing Mind and Being Mind: ['0', '0', '0', '0', '0']
Clarifying Priorities: ['0', '0', '0', '0', '0']
DEAR MAN: ['0', '0', '0', '0', '0']
GIVE: ['0', '0', '0', '0', '0']
FAST: ['0', '0', '0', '0', '0']
Whether and How Intensely to Ask or Say No: ['0', '0', '0', '0', '0']
Finding Potential Friends: ['0', '0', '0', '0', '0']
Mindfulness of Others: ['0', '0', '0', '0', '0']
How to End Relationships: ['0', '0', '0', '0', '0']
Dialectics: ['0', '0', '0', '0', '0']
Validation: ['0', '0', '0', '0', '0']
Behavior Change Strategies: ['0', '0', '0', '0', '0']
Understanding and Naming Emotions: ['0', '0', '0', '0', '0']
Checking the Facts: ['0'

In [ ]:
async def get_skill_presence_response(client, sys_prompt, eval_msgs, n):
    return await client.chat.completions.create(
        model='gpt4-1106',
        # model="gpt-35-turbo",
        messages=[
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": eval_msgs},
        ],
        n=n,
        temperature=0.,
    )

async def get_skill_presences(client, sys_prompt_base, eval_msgs, n):
    skill_names = list(dbt_skills_ref.skill_name)[:]
    skill_descs = list(dbt_skills_ref.skill_desc)[:]
    sys_prompts = [sys_prompt_base.replace("<SKILL_DESC>", desc) for desc in skill_descs]
    tasks = [get_skill_presence_response(client, sys_prompt, eval_msgs, n) for sys_prompt in sys_prompts]
    responses = await asyncio.gather(*tasks)
    for i, skill_name in enumerate(skill_names):
        ratings = [choice.message.content for choice in responses[i].choices]
        print(f'{skill_name}: {ratings}')

t_start = time.perf_counter()
await get_skill_presences(client_async, skill_extract_base_prompt2, skill_extract_test, 5)
t_stop = time.perf_counter()
print(t_stop - t_start)

In [19]:
async def get_skill_presence_response(client, sys_prompt, eval_msgs, n):
    response = await client.chat.completions.create(
        model='gpt4-1106',
        # model="gpt-35-turbo",
        messages=[
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": eval_msgs},
        ],
        n=n,
        temperature=0.,
    )
    return [choice.message.content for choice in response.choices]

async def get_skill_presences(client, sys_prompt_base, eval_msgs, n):
    skill_names = list(dbt_skills_ref.skill_name)[:]
    skill_descs = list(dbt_skills_ref.skill_desc)[:]
    sys_prompts = [sys_prompt_base.replace("<SKILL_DESC>", desc) for desc in skill_descs]
    skill_presence_dict = dict.fromkeys(skill_names)
    for i, skill_name in enumerate(skill_names):
        skill_presence_dict[skill_name] = await asyncio.create_task(get_skill_presence_response(client, sys_prompts[i], eval_msgs, n))
        await asyncio.sleep(5)
    print(skill_presence_dict)

    # tasks = [get_skill_presence_response(client, sys_prompt, eval_msgs, n) for sys_prompt in sys_prompts]
    # responses = await asyncio.gather(*tasks)
    # for i, skill_name in enumerate(skill_names):
    #     ratings = [choice.message.content for choice in responses[i].choices]
    #     print(f'{skill_name}: {ratings}')

t_start = time.perf_counter()
await get_skill_presences(client_async, skill_extract_base_prompt2, skill_extract_test, 5)
t_stop = time.perf_counter()
print(t_stop - t_start)

{'Wise Mind': ['0', '0', '0', '0', '0'], 'Observing': ['0', '0', '0', '0', '0'], 'Describing': ['0', '0', '0', '0', '0'], 'Participating': ['0', '0', '0', '0', '0'], 'Nonjudgmentally': ['0', '0', '0', '0', '0'], 'One-Mindfully': ['0', '0', '0', '0', '0'], 'Effectively': ['0', '0', '0', '0', '0'], 'Mindfulness Practice: A Spiritual Perspective': ['0', '0', '0', '0', '0'], 'Balancing Doing Mind and Being Mind': ['0', '0', '0', '0', '0'], 'Clarifying Priorities': ['0', '0', '0', '0', '0'], 'DEAR MAN': ['0', '0', '0', '0', '0'], 'GIVE': ['0', '0', '0', '0', '0'], 'FAST': ['0', '0', '0', '0', '0'], 'Whether and How Intensely to Ask or Say No': ['0', '0', '0', '0', '0'], 'Finding Potential Friends': ['0', '0', '0', '0', '0'], 'Mindfulness of Others': ['0', '0', '0', '0', '0'], 'How to End Relationships': ['0', '0', '0', '0', '0'], 'Dialectics': ['0', '0', '0', '0', '0'], 'Validation': ['0', '0', '0', '0', '0'], 'Behavior Change Strategies': ['0', '0', '0', '0', '0'], 'Understanding and Namin

In [18]:
async def get_skill_presence_response(client, semaphore, sys_prompt, eval_msgs):
    async with semaphore:
        return await client.chat.completions.create(
            model='gpt4-1106',
            # model="gpt-35-turbo",
            messages=[
                {"role": "system", "content": sys_prompt},
                {"role": "user", "content": eval_msgs},
            ],
            temperature=0.,
        )

async def get_skill_presences(
        client:AsyncAzureOpenAI,
        max_parallel_calls:int,
        sys_prompt_base:str,
        eval_msgs:str,
        n_replications:int|None = None):
    assert max_parallel_calls > 0
    if n_replications:
        assert n_replications > 0
    skill_names = list(dbt_skills_ref.skill_name)[:5]
    skill_descs = list(dbt_skills_ref.skill_desc)[:5]
    sys_prompts = [sys_prompt_base.replace("<SKILL_DESC>", desc) for desc in skill_descs]
    semaphore = asyncio.Semaphore(value=max_parallel_calls)
    tasks = [get_skill_presence_response(client, semaphore, sys_prompt, eval_msgs) for sys_prompt in sys_prompts]
    responses = await asyncio.gather(*tasks)
    for i, skill_name in enumerate(skill_names):
        ratings = [choice.message.content for choice in responses[i].choices]
        print(f'{skill_name}: {ratings}')

t_start = time.perf_counter()
await get_skill_presences(client_async, 5, skill_extract_base_prompt2, skill_extract_test, 3)
t_stop = time.perf_counter()
print(t_stop - t_start)

Wise Mind: ['0']
Observing: ['0']
Describing: ['0']
Participating: ['0']
Nonjudgmentally: ['0']
3.857143000001088


In [19]:
skill_names = list(dbt_skills_ref.skill_name)[:5]
skill_names

['Wise Mind', 'Observing', 'Describing', 'Participating', 'Nonjudgmentally']

In [20]:
[skill_name for skill_name in skill_names for i in range(3)]

['Wise Mind',
 'Wise Mind',
 'Wise Mind',
 'Observing',
 'Observing',
 'Observing',
 'Describing',
 'Describing',
 'Describing',
 'Participating',
 'Participating',
 'Participating',
 'Nonjudgmentally',
 'Nonjudgmentally',
 'Nonjudgmentally']

In [41]:
eval_rewrite_prompt = '''
You will be given a GPT system prompt used for evaluating whether a therapist discussed a specific Dialectical Behavior Therapy (DBT) skill with their client during a session.
Rewrite and reorganize this system prompt so that it is:
- More effective in guiding GPT behavior to match the intended purpose
- Clearly worded
- More concise
'''.strip()

In [42]:
response = client.chat.completions.create(
    model="gpt4-1106",
    messages=[
        {"role": "system", "content": eval_rewrite_prompt},
        {"role": "user", "content": skill_extract_base_prompt}
    ],
    n=5,
)
ratings = [choice.message.content for choice in response.choices]
for choice in response.choices:
    print(choice.message.content, end='\n\n'+'-'*30+'\n\n')

Determine if a therapist discussed a specific Dialectical Behavior Therapy (DBT) skill during a session:

DBT Skill: <SKILL_DESC>

Review the transcript between the therapist and client. Rate whether the therapist discussed the DBT skill using the following criteria:

Rate "1" for "present" if the therapist:
- Named the skill.
- Explained or defined the skill.
- Instructed or educated the client about the skill.
- Supported the client in practicing the skill.
- Aided the client in applying the skill to situations.

Rate "0" for "not present" if:
- The skill was not mentioned.
- The skill was not explained or defined.
- The skill was not taught or covered.
- The client was not supported in practicing the skill.
- The client received no help in applying the skill.

Provide your response as a single number, either "1" or "0". No justification or additional comments are needed.

------------------------------

Evaluate whether a DBT skill was discussed by a therapist as described by <SKILL

In [117]:
regex_prompt = r'''
Act as an expert in natural language processing who is familiar with Perl-style regular expressions.
You will be given a description of a Dialectical Behavior Therapy (DBT) skill.
Given this DBT skill description, come up with a few regular expressions which would reliably allow you to determine whether this DBT skill was discussed during a therapy session transcript.

Example Input:
- The "TIP" skill in the Distress Tolerance module of DBT stands for Temperature, Intense exercise, Paced breathing, and Paired muscle relaxation.
- It is designed to rapidly alter body chemistry to reduce intense emotional arousal by engaging the parasympathetic nervous system and calming the sympathetic nervous system.
- In a session, this skill might be identified by a therapist instructing a client to hold their breath and then plunge their face into cold water (Temperature), engage in a brief burst of intense physical activity (Intense exercise), practice slow, deep breaths (Paced breathing), or tense and then relax their muscles in sequence (Paired muscle relaxation).

Example Responses:
```
.*\bTIP{1,2}\b.*
```

```
.*\b[Tt]emperature\b.*\b[Ii]ntense [Ee]xercise\b.*\b[Pp]aced [Bb]reath(ing|s)?\b.*\b[Pp]aired [Mm]uscle [Rr]elaxation\b.*
```
'''.strip()

In [120]:
regex_skill_desc = '''
- The "FAST" skill in the Interpersonal Effectiveness module of DBT focuses on maintaining self-respect during interactions with others.
- FAST stands for (be) Fair, (no) Apologies for being alive or for having an opinion, Stick to values, and (be) Truthful.
- This skill is used to help individuals assert their needs and opinions without compromising their own values and integrity.
- In a session, this skill would be identified by the therapist guiding the client through one or more of the steps in "FAST."
'''.strip()

In [121]:
response = client.chat.completions.create(
    model="gpt4-1106",
    messages=[
        {"role": "system", "content": regex_prompt},
        {"role": "user", "content": regex_skill_desc}
    ],
)
print(response.choices[0].message.content)

Based on the description provided for the "FAST" skill in DBT, you would want to create regular expressions that capture any mention or discussion of these concepts within a therapy session transcript. Here are a few regular expressions that could help identify whether the "FAST" skill was discussed:

1. To match the acronym "FAST" with variations in case:
```
.*\bFAST\b.*
.*\bF(?i)ast\b.*
```

2. To match each component of the "FAST" skill with some allowance for variations in phrasing:
```
.*\b(be )?F(?i)air\b.*
.*\b(no )?[Aa]pologies\b.*(alive|opinion)\b.*
.*\b[Ss]tick\b.*\b[Tt]o\b.*\b[Vv]alues\b.*
.*\b(be )?[Tt]ruthful\b.*
```

3. A comprehensive regular expression to look for any mention of all four components of FAST in no specific order:
```
.*\b(be )?F(?i)air\b.*|(.*\b(no )?[Aa]pologies\b.*(alive|opinion)\b.*)|(.*\b[Ss]tick\b.*\b[Tt]o\b.*\b[Vv]alues\b.*)|(.*\b(be )?[Tt]ruthful\b.*)
```

4. A regular expression to search for any mention of a therapist guiding a client which migh

In [33]:
print(skill_extract_test)

Therapist: Hello and welcome! I'm glad you're here to explore Dialectical Behavior Therapy (DBT) skills. Is there a particular DBT skill or technique you're interested in practicing today, or would you like me to suggest one based on what you're experiencing currently?

Client: My grandmother passed away two days ago and everyone in my family has been busy grieving so I've been picking up all the preparations for her wake. I haven't had a chance to stop but if I don't take care of everything, I don't think anyone else will.

Therapist: I'm so sorry to hear about your loss. It sounds like you’re carrying a heavy burden during this tough time. Grief can be overwhelming, and it's important to take care of yourself too. How are you currently feeling in this moment as you handle these responsibilities?

Client: Thank you for your kind words. I'm feeling incredibly overwhelmed and exhausted. It's like I'm running on autopilot, trying to keep it all together for my family's sake, but inside I

In [141]:
pattern_str = r'.*(\bbe(ing)?\b\W*)?\b[Ff]air\b.*(\bno\b\W*)?\b[Aa]polog(y|ies|izing)\b.*\b[Ss]tick(ing|s)?\b(\W*\bto values\b)?.*(\bbe(ing)?\b\W*)?\b[Tt]ruthful\b.*'
pattern = re.compile(pattern_str, re.S)
match = re.match(pattern, regex_skill_desc)
match

<re.Match object; span=(0, 511), match='- The "FAST" skill in the Interpersonal Effective>

In [142]:
match.groups()

(None, None, None, 'ies', None, ' to values', None, None)

In [135]:
pattern_str = r'.*(\bbe\b)?\b[Ff]air\b.*'
test_str = 'be fair to me'
pattern = re.compile(pattern_str, re.S)
match = re.match(pattern, test_str)
match

<re.Match object; span=(0, 13), match='be fair to me'>

In [138]:
match.groups()

(None,)